使用torch训练 MNISET 数据集，MLP 选择： `784 → 256 → 128 → 10`

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# 1. 数据预处理
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_dataset = datasets.MNIST(root="./data", train=True, download=True, transform=transform)
test_dataset  = datasets.MNIST(root="./data", train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=1000, shuffle=False)


# 2. 定义模型：784→256→128→10
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(784, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 10)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = x.view(-1, 784)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x


# 3. 初始化
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MLP().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()


# 4. 训练函数
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)

        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        if batch_idx % 100 == 0:
            print(f"Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} "
                  f"({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}")


# 5. 测试函数
def test():
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader)
    acc = 100. * correct / len(test_loader.dataset)
    print(f"\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({acc:.2f}%)\n")


# 6. 保存/加载函数
def save_model(path="mlp_mnist.pth"):
    torch.save(model.state_dict(), path)
    print(f"Model saved to {path}")

def load_model(path="mlp_mnist.pth"):
    state_dict = torch.load(path, map_location=device)
    model.load_state_dict(state_dict)
    print(f"Model loaded from {path}")


# 7. 主循环
for epoch in range(1, 6):
    train(epoch)
    test()

# 训练完成后保存模型
save_model()

# （示例：加载模型再测试一次）
# load_model()
# test()


100.0%
100.0%
100.0%
100.0%


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.299391
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.606432
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.322016
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.124803
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.073168
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.160057
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.103431
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.144202
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.237552
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.136156

Test set: Average loss: 0.1059, Accuracy: 9683/10000 (96.83%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.043966
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.072673
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.079310
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.085021
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.061136
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.245353
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.164497
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.058751
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.18747